# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
#!pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
%config Completer.use_jedi = False

In [3]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment

############### Change file_name argument to your path ########################
env = UnityEnvironment(file_name=r'C:\Users\Ai\Documents\openAI\DeepRL_P2\Reacher_Windows_x86_64\Reacher.exe')
#env = UnityEnvironment(file_name='/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64') # for Linux

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name=r'C:\Users\Ai\Documents\openAI\DeepRL_P2\Reacher_Windows_x86_64_20\Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [6]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.0


When finished, you can close the environment.

In [7]:
#env.close()

### 4. Build an Agent

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device
device = "cpu"

##### 4.1. Policy Network architecture

In [10]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

seed = torch.manual_seed(0)
np.random.seed(0)
   
class ActorNN(nn.Module):
    
    def __init__(self, state_size, action_size):
        super(ActorNN, self).__init__()
        self.fc1 = nn.Linear(state_size, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, action_size)
        

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.tanh(self.fc3(x))
        return x
    
    
class CriticNN(nn.Module):
    
    def __init__(self, state_size):
        super(CriticNN, self).__init__()
        self.fc1 = nn.Linear(state_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


class PolicyNN(nn.Module):
    
    def __init__(self, state_size, action_size):
        super(PolicyNN, self).__init__()
        self.actor_policy = ActorNN(state_size, action_size)
        self.critic_value = CriticNN(state_size)
        # for action sampling in buffer and action log probability calculation
        self.std = nn.Parameter(torch.ones(1, action_size))
        self.to(device)

    def forward(self, states, action=None):
        states = torch.Tensor(states)
        a = self.actor_policy(states)
        v = self.critic_value(states)
        
        dist = torch.distributions.Normal(a, self.std)
        if action is None: # action sampling
            action = dist.sample()
        log_prob = dist.log_prob(action)
        log_prob = torch.sum(log_prob, dim=1, keepdim=True)
        return action, log_prob, v

##### 4.2. Helper object for batch optimization

In [11]:
class Batcher:
    
    def __init__(self, batch_size, data):
        self.batch_size = batch_size
        self.data = data
        self.num_entries = len(data[0])
        self.reset()

    def reset(self):
        self.batch_start = 0
        self.batch_end = self.batch_start + self.batch_size

    def end(self):
        return self.batch_start >= self.num_entries

    def next_batch(self):
        batch = []
        for d in self.data:
            batch.append(d[self.batch_start: self.batch_end])
        self.batch_start = self.batch_end
        self.batch_end = min(self.batch_start + self.batch_size, self.num_entries)
        return batch

    def shuffle(self):
        indices = np.arange(self.num_entries)
        np.random.shuffle(indices)
        self.data = [d[indices] for d in self.data]

##### 4.3. The Agent - Proximal Policy Optimization using Actor and Critic

In [12]:
#from IPython.core.debugger import set_trace

In [13]:
import numpy as np
#import random
import torch
import torch.nn as nn


class PPOAgent(object):
    
    def __init__(self, environment, brain_name, policy_network, optimizer, hyperparameters):

        self.network = policy_network
        self.optimizer = optimizer
        self.hyperparameters = hyperparameters
        
        self.environment = environment
        self.brain_name = brain_name        
        env_info = environment.reset(train_mode=True)[brain_name]
        self.num_agents = len(env_info.agents)
        self.states = env_info.vector_observations
        self.ploss = []
        self.vloss = []
        self.step_count = 0

    def step(self):
        self.step_count += 1
        buffer = [] # on-policy buffer filling
        hyperparameters = self.hyperparameters
        
        # fill critic's memory with policy samples
        env_info = self.environment.reset(train_mode=True)[self.brain_name]    
        self.states = env_info.vector_observations  
        states = self.states
        
        self.network.eval()
        with torch.no_grad():
            for _ in range(hyperparameters['buffer_size']):
                actions, log_probs, values = self.network(states)
                env_info = self.environment.step(actions.cpu().detach().numpy())[self.brain_name]
                next_states = env_info.vector_observations
                rewards = env_info.rewards
                dones = np.array([1 if t else 0 for t in env_info.local_done])

                buffer.append([states, values.detach(), actions.detach(), log_probs.detach(), rewards, 1 - dones])
                states = next_states

            self.states = states
            pending_value = self.network(states)[-1]
            buffer.append([states, pending_value, None, None, None, None])
        self.network.train()
        
        # process buffer and calculate advantages for learning 
        processed_buffer = [None] * (len(buffer) - 1)
        advantages = torch.Tensor(np.zeros((self.num_agents, 1)))
        returns = pending_value.detach()
        for i in reversed(range(len(buffer) - 1)): # reversed
            states, value, actions, log_probs, rewards, dones = buffer[i]
 
            states = torch.Tensor(states)
            actions = torch.Tensor(actions)
            rewards = torch.Tensor(rewards).unsqueeze(1)
            dones = torch.Tensor(dones).unsqueeze(1)

            next_value = buffer[i + 1][1]
            returns = rewards + hyperparameters['discount_rate'] * dones * returns
            # advantage calculation based on critic values
            td_estimate = rewards + hyperparameters['discount_rate'] * dones * next_value.detach() - value.detach() * dones
            processed_buffer[i] = [states, actions, log_probs, returns, td_estimate]

        states, actions, log_probs_critic, returns, advantages = map(lambda x: torch.cat(x, dim=0), zip(*processed_buffer))
        advantages = (advantages - advantages.mean()) / advantages.std() # normalize advantages
        #set_trace()
        # batch policy optimization (learning)
        batcher = Batcher(states.size(0) // hyperparameters['batch_size'], [np.arange(states.size(0))])
        for i in range(hyperparameters['optimization_epochs']):
            batcher.shuffle()
            while not batcher.end():
                batch_indices = batcher.next_batch()[0]
                batch_indices = torch.Tensor(batch_indices).long()
                sampled_states = states[batch_indices]
                sampled_actions = actions[batch_indices]
                sampled_log_probs = log_probs_critic[batch_indices]
                sampled_returns = returns[batch_indices]
                sampled_advantages = advantages[batch_indices]
                
                _, log_probs, values = self.network(sampled_states, sampled_actions)
                ratio = (log_probs - sampled_log_probs).exp()      # re-weighting factor
                obj = ratio * sampled_advantages
                obj_clipped = ratio.clamp(1.0 - hyperparameters['ppo_clip'],
                                          1.0 + hyperparameters['ppo_clip']) * sampled_advantages
                policy_loss = -torch.min(obj, obj_clipped).mean(0) # ppo surrogate function
                #self.ploss.append(policy_loss.detach()) # enable if you want to track policy_loss

                value_loss = (10/(self.step_count)) * (sampled_returns - values).pow(2).mean()   # initially helps to train policy
                #self.vloss.append(value_loss.detach())  # enable if you want to track value_loss

                self.optimizer.zero_grad()
                (policy_loss + value_loss).backward()
                nn.utils.clip_grad_norm_(self.network.parameters(), hyperparameters['gradient_clip'])
                self.optimizer.step()

##### 4.4. Training routine

In [14]:
def score_episode(env, brain_name, policy, num_agents):
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations     
    scores = np.zeros(num_agents)
    while True:
        actions, _, _ = policy(states)
        env_info = env.step(actions.cpu().detach().numpy())[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        scores += env_info.rewards
        states = next_states
        if np.any(dones):
            break
    
    return np.mean(scores)
    
def train(env, brain_name, policy, hyperparameters):
    optimizer = optim.Adam(policy.parameters(), hyperparameters['adam_learning_rate'],
                    eps=hyperparameters['adam_epsilon'])
    agent = PPOAgent(env, brain_name, policy, optimizer, hyperparameters)
    history = []
    running_averages = []
    goal_score = 30.0

    for i in range(hyperparameters['episode_count']):
        agent.step()
        last_mean_reward = score_episode(env, brain_name, policy, agent.num_agents)
        history.append(last_mean_reward)
        running_average = np.mean(np.array(history[-100:]))
        running_averages.append(running_average)
        if running_average > goal_score:
            torch.save(policy.state_dict(), "ppo-solved.pth")
            break
        print('Episode: {:3.0f} Score: {:5.2f} vs. average: {:5.2f} ({:+.2f})'.format(i + 1, last_mean_reward, running_average, (last_mean_reward-running_average)))
    return history, running_averages, agent.ploss, agent.vloss



In [15]:
hyperparameters = {
        'episode_count': 1500,
        'discount_rate': 0.95,
        'gradient_clip': 15,
        'buffer_size': 3072,
        'optimization_epochs': 2,
        'ppo_clip': 0.2,
        'batch_size': 512,
        'adam_learning_rate': 3e-4,
        'adam_epsilon': 1e-4
    }

##### This section optionally can be skipped to 5. Test Agent

In [16]:
###### takes some time to train ######


policy = PolicyNN(state_size, action_size)
history, average_scores, ploss, vloss = train(env, brain_name, policy, hyperparameters)

Episode:   1 Score:  0.44 vs. average:  0.44 (+0.00)
Episode:   2 Score:  0.13 vs. average:  0.28 (-0.15)
Episode:   3 Score:  1.16 vs. average:  0.58 (+0.58)
Episode:   4 Score:  0.50 vs. average:  0.56 (-0.06)
Episode:   5 Score:  1.13 vs. average:  0.67 (+0.46)
Episode:   6 Score:  0.00 vs. average:  0.56 (-0.56)
Episode:   7 Score:  0.77 vs. average:  0.59 (+0.18)
Episode:   8 Score:  0.20 vs. average:  0.54 (-0.34)
Episode:   9 Score:  0.58 vs. average:  0.55 (+0.03)
Episode:  10 Score:  0.55 vs. average:  0.55 (+0.00)
Episode:  11 Score:  1.79 vs. average:  0.66 (+1.13)
Episode:  12 Score:  2.77 vs. average:  0.83 (+1.93)
Episode:  13 Score:  2.08 vs. average:  0.93 (+1.15)
Episode:  14 Score:  1.91 vs. average:  1.00 (+0.91)
Episode:  15 Score:  1.63 vs. average:  1.04 (+0.59)
Episode:  16 Score:  1.93 vs. average:  1.10 (+0.83)
Episode:  17 Score:  1.44 vs. average:  1.12 (+0.32)
Episode:  18 Score:  0.82 vs. average:  1.10 (-0.28)
Episode:  19 Score:  1.65 vs. average:  1.13 (

Episode: 155 Score: 38.48 vs. average: 23.47 (+15.01)
Episode: 156 Score: 37.21 vs. average: 23.68 (+13.53)
Episode: 157 Score: 33.98 vs. average: 23.81 (+10.17)
Episode: 158 Score: 26.89 vs. average: 23.99 (+2.90)
Episode: 159 Score: 36.22 vs. average: 24.16 (+12.06)
Episode: 160 Score: 39.11 vs. average: 24.42 (+14.69)
Episode: 161 Score: 34.07 vs. average: 24.55 (+9.52)
Episode: 162 Score: 39.32 vs. average: 24.74 (+14.58)
Episode: 163 Score: 36.09 vs. average: 24.98 (+11.11)
Episode: 164 Score: 33.01 vs. average: 25.10 (+7.91)
Episode: 165 Score: 37.77 vs. average: 25.37 (+12.40)
Episode: 166 Score: 38.10 vs. average: 25.52 (+12.58)
Episode: 167 Score: 39.24 vs. average: 25.67 (+13.57)
Episode: 168 Score: 38.80 vs. average: 25.72 (+13.08)
Episode: 169 Score: 37.12 vs. average: 25.89 (+11.23)
Episode: 170 Score: 33.07 vs. average: 26.14 (+6.93)
Episode: 171 Score: 35.65 vs. average: 26.36 (+9.29)
Episode: 172 Score: 38.17 vs. average: 26.53 (+11.64)
Episode: 173 Score: 39.49 vs. ave

##### Loss tracking (This section optionally can be skipped to 5. Test Agent)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(ploss)), ploss)
plt.ylabel('Policy Loss')
plt.xlabel('Step #')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(vloss)), vloss)
plt.ylabel('Value Loss')
plt.xlabel('Step #')
plt.show()

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(np.vstack([np.array(vloss), np.array(ploss)]).T, columns=['vloss', 'ploss'])
df

In [ ]:
df.to_pickle('AY.df')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(vloss)), np.array(vloss)+np.array(ploss))
plt.ylabel('Total Loss')
plt.xlabel('Step #')
plt.show()
plt.show()

In [ ]:
#env.close()

##### 4.5. Plot scores history

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(history)), history)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Test the trained Agent

In [ ]:
policy = PolicyNN(state_size, action_size)
policy.load_state_dict(torch.load('ppo-solved.pth'))

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment (train_mode=False shows an agent in action)
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions, _, _ = policy(states)                     # select an action (for each agent)
    env_info = env.step(actions.cpu().detach().numpy())[brain_name]          # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
    
print("Score: {:.2f}".format(np.mean(scores)))         # print the score (of a single agent)

In [ ]:
env.close()